In [ ]:
# 1. 라이브러리 불러오기
import pandas as pd                     # 데이터프레임 처리
import numpy as np                      # 수치 계산
import matplotlib.pyplot as plt         # 시각화
from statsmodels.tsa.statespace.sarimax import SARIMAX  # SARIMA 모델
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  # ACF, PACF 시각화
from statsmodels.tsa.stattools import adfuller  # ADF 테스트 (정상성 검정)
from statsmodels.stats.diagnostic import acorr_ljungbox  # 잔차 진단 (Ljung-Box 테스트)
from sklearn.metrics import mean_squared_error  # 모델 평가 지표 (MSE)

# 2. 데이터 로딩 및 전처리
data = pd.read_csv('통합된_수출_예측_데이터셋.csv', encoding='CP949')  # CSV 파일 읽기 (한글 인코딩 주의) // **EUC-KR(한글 완성형)**을 기반으로 한 Windows 전용 한글 인코딩 방식
#data = pd.read_excel('통합된_수출_예측_데이터셋.xlsx', encoding='CP949') # 데이터 통합본 넣기
df = pd.DataFrame(data)  # DataFrame으로 변환

# '날짜' 열을 datetime 형식으로 변환 후 인덱스로 설정
df['날짜'] = pd.to_datetime(df['날짜'], format='%Y-%m')
df.set_index('날짜', inplace=True)

# 시계열 데이터 시각화
plt.figure(figsize=(10, 5))
plt.plot(df.index, df['수출량'], marker='o')
plt.title('월별 수출량')
plt.grid()
plt.show()

# 3. 정상성 확인 (ADF Test)
adf_result = adfuller(df['수출량'])  # ADF 검정을 통해 정상성 여부 확인
print(f'ADF Statistic: {adf_result[0]}')  # 검정 통계량
print(f'p-value: {adf_result[1]}')       # p-value
for key, value in adf_result[4].items(): # 임계값 출력
    print(f'Critical Values {key}: {value}')

# 4. 1차 차분(Differencing)
df['수출량_diff'] = df['수출량'].diff().dropna()  # 1차 차분으로 비정상 시계열을 정상으로 변환

# 차분된 시계열 시각화
plt.figure(figsize=(10, 5))
plt.plot(df.index, df['수출량_diff'])
plt.title('Differenced Time Series')
plt.grid()
plt.show()

# 5. ACF 및 PACF 분석 (모수 결정 보조 도구)
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
plot_acf(df['수출량_diff'].dropna(), ax=axes[0])  # 자기상관 함수
axes[0].set_title('ACF Plot')
plot_pacf(df['수출량_diff'].dropna(), ax=axes[1])  # 부분 자기상관 함수
axes[1].set_title('PACF Plot')
plt.show()

# 6. SARIMA 모델 생성 및 학습
model = SARIMAX(df['수출량'],
                order=(2, 1, 1),             # ARIMA의 (p,d,q)
                seasonal_order=(1, 1, 1, 12))  # 계절성 있는 SARIMA의 (P,D,Q,s) s=12개월
model_fit = model.fit()  # 모델 학습

# 학습된 모델의 요약 출력
print(model_fit.summary())

# 7. 잔차 분석 (Ljung-Box Test: 잔차가 백색잡음인지 확인)
print('========================')
residuals = model_fit.resid  # 잔차 추출
ljung_box_result = acorr_ljungbox(residuals, lags=[10], return_df=True)  # 10시차까지 검정
print(ljung_box_result)
print('========================')

# 잔차 시각화 (패턴이 없고 평균 주변을 중심으로 움직이면 좋음)
plt.figure(figsize=(10, 5))
plt.plot(residuals)
plt.title('Residuals from SARIMA Model')
plt.grid()
plt.show()

# 8. 예측 수행 및 시각화
forecast = model_fit.forecast(steps=12)  # 향후 12개월 예측

# 예측값 출력
print("Forecasted Values:")
print(forecast)

# 예측 결과 시각화
plt.figure(figsize=(12, 5))
plt.plot(df.index, df['수출량'], label='Original')  # 실제 데이터
# 예측 구간은 마지막 시점 다음 달부터 12개월 후까지
plt.plot(pd.date_range(start=df.index[-1], periods=13, freq='M')[1:], forecast, label='Forecast', linestyle='--')
plt.title('SARIMA Forecast')
plt.legend()
plt.grid()
plt.show()

# 9. 모델 평가 (MSE, RMSE)
# 마지막 12개월 실제값과 예측값 비교 → 성능 평가 (주의: 실제 마지막 12개월 예측한 것이 아님)
mse = mean_squared_error(df['수출량'][-12:], forecast)
rmse = np.sqrt(mse)

print('==================================================')
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print('==================================================')
